> Scrape this Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [1]:
import pandas as pd # library for data analysis
import numpy as np # library to handle data in a vectorized manner

In [2]:
#conda install -c anaconda lxml

In [3]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

In [4]:
df=dfs[0]
df.rename(columns={'Postcode':'PostalCode'}, inplace=True) #The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
print (df.shape) 
df.head()

(288, 3)


,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


> Ignore cells with a borough that is Not assigned.

In [5]:
df = df[df.Borough != "Not assigned"]
print (df.shape) 
df.head()

(211, 3)


,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


> notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
> These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [6]:
df = df.groupby('PostalCode').agg({ 'Borough': 'first', 'Neighbourhood':', '.join }).reset_index()
print (df.shape) 
df.head()

(103, 3)


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


> If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [8]:
df[df.Neighbourhood == "Not assigned"]

,PostalCode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [9]:
df.loc[df.Neighbourhood == "Not assigned", 'Neighbourhood'] = df.Borough
df[df.Neighbourhood == "Not assigned"] # check if it is still there

,PostalCode,Borough,Neighbourhood


In [10]:
df[df.PostalCode == "M7A"] #confirm replacement 

,PostalCode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [11]:
df.shape

(103, 3)

In [14]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

In [15]:
#conda install -y -c conda-forge geocoder

In [48]:
import geocoder # import geocoder

In [49]:
# initialize your variable to None
lat_lng_coords = None
postal_code = "M5G"
i=0

# loop until you get the coordinates
while(lat_lng_coords is None):
    i = i + 1
    print(i, '{}, Toronto, Ontario'.format(postal_code))
    if i>5:
        break
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

if(lat_lng_coords):
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    print("lat ",latitude," lon ",longitude)

#geocoder does not produce results, will import CSV

1 M5G, Toronto, Ontario
2 M5G, Toronto, Ontario
3 M5G, Toronto, Ontario
4 M5G, Toronto, Ontario
5 M5G, Toronto, Ontario
6 M5G, Toronto, Ontario


> link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [16]:
dfcodes = pd.read_csv('http://cocl.us/Geospatial_data', header=0)
print(dfcodes.shape)
dfcodes.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
dfmerge = pd.concat([df, dfcodes], axis=1)
dfmerge.head()

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [18]:
dfmerge=pd.DataFrame(dfmerge,columns=['PostalCode','Borough','Neighbourhood','Latitude','Longitude'])
dfmerge.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


> Explore and cluster the neighborhoods in Toronto. 

In [19]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [23]:
neighborhoods=dfmerge
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [47]:
neighborhoods[neighborhoods['Longitude']=='']

,PostalCode,Borough,Neighbourhood,Latitude,Longitude


Map of Toronto 

In [21]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [22]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [34]:
# create map of Toronto using latitude and longitude values
map_1 = folium.Map(location=[latitude, longitude], zoom_start=11, width=800, height=700, tiles='Stamen Toner')

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,        
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_1)  
    
map_1

In [41]:
#Foursquare settings
CLIENT_ID = 'IEELNVNQLNIMZOIGZZ0XTT2VFNIQ5SCCTJBZIHAJXJV5WMCM' # your Foursquare ID
CLIENT_SECRET = 'EI4ABXOSKG0DNPFBPPK133PMOQMWJYMUMB1Q5ZXO42WY4PUS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

Your credentails:
CLIENT_ID: IEELNVNQLNIMZOIGZZ0XTT2VFNIQ5SCCTJBZIHAJXJV5WMCM
CLIENT_SECRET:EI4ABXOSKG0DNPFBPPK133PMOQMWJYMUMB1Q5ZXO42WY4PUS


> Function to find venues per neighbourhood

In [49]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    i=0
    for name, lat, lng in zip(names, latitudes, longitudes):
        i=i+1
        print(i, name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        if(results):
            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [50]:
toronto_neigh = getNearbyVenues(names=neighborhoods['Neighbourhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

1 Rouge, Malvern
2 Highland Creek, Rouge Hill, Port Union
3 Guildwood, Morningside, West Hill
4 Woburn
5 Cedarbrae
6 Scarborough Village
7 East Birchmount Park, Ionview, Kennedy Park
8 Clairlea, Golden Mile, Oakridge
9 Cliffcrest, Cliffside, Scarborough Village West
10 Birch Cliff, Cliffside West
11 Dorset Park, Scarborough Town Centre, Wexford Heights
12 Maryvale, Wexford
13 Agincourt
14 Clarks Corners, Sullivan, Tam O'Shanter
15 Agincourt North, L'Amoreaux East, Milliken, Steeles East
16 L'Amoreaux West
17 Upper Rouge
18 Hillcrest Village
19 Fairview, Henry Farm, Oriole
20 Bayview Village
21 Silver Hills, York Mills
22 Newtonbrook, Willowdale
23 Willowdale South
24 York Mills West
25 Willowdale West
26 Parkwoods
27 Don Mills North
28 Flemingdon Park, Don Mills South
29 Bathurst Manor, Downsview North, Wilson Heights
30 Northwood Park, York University
31 CFB Toronto, Downsview East
32 Downsview West
33 Downsview Central
34 Downsview Northwest
35 Victoria Village
36 Woodbine Gardens, P

In [53]:
print("We have",toronto_neigh.shape[0],"venues")
toronto_neigh.head()

We have 2242 venues


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [55]:
#venues per neighbourhood
toronto_neigh.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",9,9,9,9,9,9
...,...,...,...,...,...,...
Willowdale West,6,6,6,6,6,6
Woburn,4,4,4,4,4,4
"Woodbine Gardens, Parkview Hill",13,13,13,13,13,13


In [56]:
print('There are {} uniques categories.'.format(len(toronto_neigh['Venue Category'].unique())))

There are 271 uniques categories.


# Analysis

In [112]:
# one hot encoding
toronto_neigh_onehot = pd.get_dummies(toronto_neigh[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_neigh_onehot['Neighbourhood'] = toronto_neigh['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_neigh_onehot.columns[-1]] + list(toronto_neigh_onehot.columns[:-1])
toronto_neigh_onehot = toronto_neigh_onehot[fixed_columns]

print(toronto_neigh_onehot.shape)
toronto_neigh_onehot.head()

(2242, 272)


,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [113]:
#frequency of venues 
toronto_grouped = toronto_neigh_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03,...,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0


In [114]:
#function for most common venues 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [155]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd'] 

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,American Restaurant,Burger Joint,Asian Restaurant,Thai Restaurant,Sushi Restaurant,Restaurant,Bakery
1,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Coffee Shop,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Grocery Store,Pharmacy,Fried Chicken Joint,Sandwich Place,Discount Store,Beer Store,Japanese Restaurant,Fast Food Restaurant,German Restaurant
4,"Alderwood, Long Branch",Pizza Place,Pharmacy,Skating Rink,Coffee Shop,Pool,Pub,Sandwich Place,Gym,Airport Terminal,Farmers Market


# Clustering

In [156]:
#run k-means to create 7 clusters 

# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([2, 4, 0, 4, 2, 2, 4, 2, 2, 4, 2, 2, 4, 0, 4, 2, 0, 2, 4, 2],
      dtype=int32)

In [157]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

In [158]:
print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(103, 16)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,4.0,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dessert Shop
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,5.0,Bar,History Museum,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4.0,Mexican Restaurant,Pizza Place,Electronics Store,Breakfast Spot,Intersection,Medical Center,Rental Car Location,Yoga Studio,Discount Store,Dog Run
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2.0,Coffee Shop,Korean Restaurant,Insurance Office,Event Space,Ethiopian Restaurant,Falafel Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4.0,Bakery,Bank,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Donut Shop,Discount Store,Dog Run


In [159]:
print(toronto_merged[toronto_merged['Cluster Labels'].isnull()])

   PostalCode      Borough             Neighbourhood   Latitude  Longitude  \
16        M1X  Scarborough               Upper Rouge  43.836125 -79.205636   
20        M2L   North York  Silver Hills, York Mills  43.757490 -79.374714   
21        M2M   North York   Newtonbrook, Willowdale  43.789053 -79.408493   
93        M9A    Etobicoke          Islington Avenue  43.667856 -79.532242   
98        M9N         York                    Weston  43.706876 -79.518188   

    Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
16             NaN                   NaN                   NaN   
20             NaN                   NaN                   NaN   
21             NaN                   NaN                   NaN   
93             NaN                   NaN                   NaN   
98             NaN                   NaN                   NaN   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
16                   NaN                   NaN                

In [160]:
#clean NaN values 
toronto_clean = toronto_merged[np.isfinite(toronto_merged['Cluster Labels'])]
toronto_clean.shape

(98, 16)

## Show clusters on a map

In [161]:
# create map
map_clusters = folium.Map(location=[latitude, longitude],  zoom_start=11, width=800, height=700, tiles='Stamen Toner')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_clean['Latitude'], toronto_clean['Longitude'], toronto_clean['Neighbourhood'], toronto_clean['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

In [162]:
toronto_clean.loc[toronto_clean['Cluster Labels'] == 0, toronto_clean.columns[[1] + list(range(5, toronto_clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,0.0,Playground,Park,Coffee Shop,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
23,North York,0.0,Park,Bank,Convenience Store,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
25,North York,0.0,Park,Pool,Food & Drink Shop,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
30,North York,0.0,Airport,Park,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
40,East York,0.0,Park,Convenience Store,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
44,Central Toronto,0.0,Park,Bus Line,Lake,Swim School,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
50,Downtown Toronto,0.0,Park,Playground,Trail,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
74,York,0.0,Park,Fast Food Restaurant,Women's Store,Market,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
90,Etobicoke,0.0,Park,River,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
100,Etobicoke,0.0,Pizza Place,Park,Bus Line,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [163]:
toronto_clean.loc[toronto_clean['Cluster Labels'] == 1, toronto_clean.columns[[1] + list(range(5, toronto_clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Etobicoke,1.0,Golf Course,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


In [164]:
toronto_clean.loc[toronto_clean['Cluster Labels'] == 2, toronto_clean.columns[[1] + list(range(5, toronto_clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,2.0,Coffee Shop,Korean Restaurant,Insurance Office,Event Space,Ethiopian Restaurant,Falafel Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
6,Scarborough,2.0,Discount Store,Department Store,Coffee Shop,Hobby Shop,Dumpling Restaurant,Diner,Dog Run,Doner Restaurant,Donut Shop,Drugstore
10,Scarborough,2.0,Indian Restaurant,Pet Store,Furniture / Home Store,Vietnamese Restaurant,Chinese Restaurant,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
15,Scarborough,2.0,Grocery Store,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,Indian Restaurant,Pharmacy,Coffee Shop,Sandwich Place,Pizza Place,American Restaurant
18,North York,2.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Bakery,Electronics Store,Japanese Restaurant,Shopping Mall,Spa,Smoothie Shop
22,North York,2.0,Ramen Restaurant,Coffee Shop,Café,Sandwich Place,Pizza Place,Restaurant,Indonesian Restaurant,Sushi Restaurant,Ice Cream Shop,Steakhouse
24,North York,2.0,Pizza Place,Coffee Shop,Discount Store,Butcher,Pharmacy,Grocery Store,Comfort Food Restaurant,Comic Shop,Ethiopian Restaurant,Empanada Restaurant
27,North York,2.0,Gym,Beer Store,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Bubble Tea Shop,Bike Shop,Sporting Goods Shop,Sandwich Place,Japanese Restaurant
28,North York,2.0,Coffee Shop,Grocery Store,Supermarket,Sushi Restaurant,Bank,Restaurant,Deli / Bodega,Fried Chicken Joint,Frozen Yogurt Shop,Diner
29,North York,2.0,Metro Station,Caribbean Restaurant,Furniture / Home Store,Massage Studio,Bar,Coffee Shop,Miscellaneous Shop,Donut Shop,Dog Run,Doner Restaurant


In [165]:
toronto_clean.loc[toronto_clean['Cluster Labels'] == 3, toronto_clean.columns[[1] + list(range(5, toronto_clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,3.0,Baseball Field,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant
97,North York,3.0,Baseball Field,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant


In [166]:
toronto_clean.loc[toronto_clean['Cluster Labels'] == 4, toronto_clean.columns[[1] + list(range(5, toronto_clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4.0,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dessert Shop
2,Scarborough,4.0,Mexican Restaurant,Pizza Place,Electronics Store,Breakfast Spot,Intersection,Medical Center,Rental Car Location,Yoga Studio,Discount Store,Dog Run
4,Scarborough,4.0,Bakery,Bank,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Donut Shop,Discount Store,Dog Run
5,Scarborough,4.0,Cosmetics Shop,Playground,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
7,Scarborough,4.0,Bakery,Bus Line,Park,Soccer Field,Intersection,Bus Station,Metro Station,Fast Food Restaurant,Dumpling Restaurant,Drugstore
8,Scarborough,4.0,Motel,American Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Dim Sum Restaurant
9,Scarborough,4.0,College Stadium,Skating Rink,General Entertainment,Café,Empanada Restaurant,Electronics Store,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop
11,Scarborough,4.0,Middle Eastern Restaurant,Auto Garage,Breakfast Spot,Vietnamese Restaurant,Shopping Mall,Sandwich Place,Yoga Studio,Doner Restaurant,Diner,Discount Store
12,Scarborough,4.0,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
13,Scarborough,4.0,Pizza Place,Pharmacy,Thai Restaurant,Noodle House,Chinese Restaurant,Italian Restaurant,Bank,Fried Chicken Joint,Fast Food Restaurant,General Entertainment


In [167]:
toronto_clean.loc[toronto_clean['Cluster Labels'] == 5, toronto_clean.columns[[1] + list(range(5, toronto_clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,5.0,Bar,History Museum,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


In [168]:
toronto_clean.loc[toronto_clean['Cluster Labels'] == 6, toronto_clean.columns[[1] + list(range(5, toronto_clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,6.0,Home Service,Garden,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
